## Tobigs 5주차 Crawling 과제
- Selenium을 활용하여 학술연구정보서비스 RISS에서 주어진 키워드에 대한 **학위 논문**을 검색 후 **첫번째 페이지**에 존재하는 논문의 **URL**을 먼저 수집합니다. 
- 수집한 URL을 기반으로 접근하여 논문 정보 **1) 저자 2) 발행년도 3) 제목 4) 부가정보의 첫번째 항목**을 수집해주세요! 
- 주석을 읽고 주어진 코드 중간에 있는 ???와 공란을 채워주시면 됩니다!

In [16]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd

In [17]:
keyword='코로나'  #검색용 키워드
paper=[]  #결과 저장용 리스트

#### 1. 드라이버 오픈 및 Riss로 이동

In [40]:
driver = webdriver.Chrome('C:/Users/xnote/Downloads/chromedriver')  #드라이버 오픈
driver.get('http://www.riss.kr/index.do')   #riss로 이동

#### 2. 검색창에 접근하여 검색용 키워드 입력 및 검색

In [42]:
driver.find_element_by_id('query').send_keys('코로나')  # 검색창 접근 및 키워드 입력
driver.find_element_by_id('query').submit()   # 검색창에 검색용 키워드 검색

#### 3. 검색 화면 창에서 학위 논문 창으로 이동
![image](https://user-images.githubusercontent.com/69336270/129825390-d270bcca-003c-46f4-8bfd-14e4f1ed3f87.png)
  
    방법 1. 상단의 학위 논문 탭 클릭  
    방법 2. 검색 화면 창에서 학위논문 결과 옆 더보기 버튼 클릭

In [43]:
button = driver.find_element_by_xpath('//*[@id="tabMenu"]/div/ul/li[3]/a/span')
button.click()

#### 4. 주어진 키워드로 검색했을 때 학위 논문의 전체 수 구하기

![image](https://user-images.githubusercontent.com/69336270/129825100-89dd9681-4dd7-451a-804f-e0fbe437aeba.png)

In [22]:
total=driver.find_element_by_xpath('//*[@id="divContent"]/div[2]/div/div[1]/dl/dd/span/span').text  #전체 논문수
print(total,'\n')

import re
total=int(re.sub(r'[^a-zA-Z0-9]','',total))
print(total)
print(f'자료형: {type(total)}')

2,348 

2348
자료형: <class 'int'>


#### 5. 첫페이지에 존재하는 논문의 상세정보 url 수집

In [23]:
urls = []

html = driver.page_source # 페이지 소스 저장
""" html 소스코드를 BeautifulSoup을 이용해 파싱 """

if total < 10:
    # 주어진 키워드로 검색 시 전체 논문 수가 10개 미만인 경우 - 논문 첫 페이지의 논문 수 = 전체 논문 수
    for i in range(1, total+1):  
        page = '/html/body/div[1]/div[2]/div[4]/div[2]/div/div[3]/div[2]/ul/li['+str(i)+']/div[2]/p[1]/a'
        url=driver.find_element_by_xpath(page).get_attribute('href')
        urls.append(url)
        print(url)
else:
    # 주어진 키워드로 검색 시 전체 논문 수가 10개 이상인 경우 - 논문 첫 페이지의 논문 수 = 10개
    for i in range(1, 11):
        page = '/html/body/div[1]/div[2]/div[4]/div[2]/div/div[3]/div[2]/ul/li['+str(i)+']/div[2]/p[1]/a'
        url=driver.find_element_by_xpath(page).get_attribute('href')
        urls.append(url)
        print(url)

http://www.riss.kr/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=aadbf6e4370d3442
http://www.riss.kr/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=e03ab5057707462affe0bdc3ef48d419
http://www.riss.kr/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=8b2e61b36976fa93ffe0bdc3ef48d419
http://www.riss.kr/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=d0fbdbd21c73b325ffe0bdc3ef48d419
http://www.riss.kr/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=29ccc72d95b8f0dbffe0bdc3ef48d419
http://www.riss.kr/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=9a09cc27b7c5cc9dffe0bdc3ef48d419
http://www.riss.kr/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=c14e36d50f743d7dffe0bdc3ef48d419
http://www.riss.kr/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=0f44641b5e2bce2effe0bdc3ef48d419
http://www.riss.kr/search/detail/DetailView.do?p_mat_type=be54d9

#### 6-1. 저자 정보 구하기 예제
수집한 url 중 첫번째 링크의 저자 정보를 구해보자!

In [24]:
data = []  # 수집한 정보 저장용 리스트
driver.get(urls[0])   # 첫번째 링크로 이동

In [25]:
""" 저자 정보를 xpath와 .text 메소드를 통해 구해주세요! """
author=driver.find_element_by_xpath('//*[@id="thesisInfoDiv"]/div[2]/div[1]/ul/li[1]/div/p/a').text
data.append(author)
print(author)

박용성


#### 6-2. 연도 정보 구하기 예제
수집한 url 중 첫번째 링크의 연도 정보를 구해보자!

In [26]:
""" 연도 정보를 xpath와 .text 메소드를 통해 구해주세요! """
year=driver.find_element_by_xpath('//*[@id="thesisInfoDiv"]/div[2]/div[1]/ul/li[4]/div/p').text
data.append(year)
print(year)

2002


#### 6-3. 제목 정보 구하기 예제
수집한 url 중 첫번째 링크의 제목 정보를 구해보자!

In [27]:
""" 제목 정보를 xpath와 .text 메소드를 통해 구해주세요! """
title=driver.find_element_by_xpath('//*[@id="thesisInfoDiv"]/div[1]/h3').text
data.append(title)
print(title)

코로나방전을 이용한 NO_(2) 전환 및 디젤엔진 Soot 산화에 관한 기초 연구 = Fundamental study on NO-NO_(2) conversion characteristics by corona discharge and oxidation of soot from diesel engine exhaust


#### 6-4. 부가정보 정보 구하기 예제
수집한 url 중 첫번째 링크의 부가정보 정보를 구해보자!

![image](https://user-images.githubusercontent.com/69336270/129768125-b13b9554-7dba-44e0-aa27-07b901a7f50c.png)
  
부가정보에서 데이터를 수집하기 위해서는 우선 더보기 버튼을 클릭해주어야 한다. 이때, 더보기 버튼의 **클래스명**을 통해 클릭해주자!

In [31]:
data

['박용성',
 '2002',
 '코로나방전을 이용한 NO_(2) 전환 및 디젤엔진 Soot 산화에 관한 기초 연구 = Fundamental study on NO-NO_(2) conversion characteristics by corona discharge and oxidation of soot from diesel engine exhaust']

In [34]:
# 부가정보 가져오기 - 더보기 버튼 클릭

moreViews = driver.find_elements_by_class_name('moreView')

for moreView in moreViews:
    moreView.send_keys(Keys.ENTER)
    sleep(0.1)

In [44]:
moreViews

[<selenium.webdriver.remote.webelement.WebElement (session="dd596885254610be5d12762a3c6e4ded", element="ab95f64a-8fca-4bb5-8674-5f54c5a70fe7")>,
 <selenium.webdriver.remote.webelement.WebElement (session="dd596885254610be5d12762a3c6e4ded", element="bac446b6-3325-4192-ac59-681c6884e13b")>,
 <selenium.webdriver.remote.webelement.WebElement (session="dd596885254610be5d12762a3c6e4ded", element="331bd317-14fd-4ee0-8de6-53e6b0a47f9c")>,
 <selenium.webdriver.remote.webelement.WebElement (session="dd596885254610be5d12762a3c6e4ded", element="93ef3027-1b38-4f76-a4fe-7b1e4bb5e06d")>,
 <selenium.webdriver.remote.webelement.WebElement (session="dd596885254610be5d12762a3c6e4ded", element="5af3b4d3-4d41-46b6-b883-e02b13bc66da")>]

In [35]:
# 부가정보 가져오기 - 첫번째 항목만 가져오기

try:
    """ 부가정보의 첫번째 항목을 xpath와 .text 메소드를 통해 구해주세요! """
    first_info=driver.find_element_by_xpath('//*[@id="additionalInfoDiv"]/div/div[1]/div[3]/p').text
    data.append(first_info)
    print(first_info)
except:
    data.append("부가정보가 없습니다.")

현재 인류의 주된 에너지원은 화석연료(C_(n)H_(m))를 대기중의 산소(O_(2))로 연소시켜 나오는 CO_(2)와 H_(2)O 가스에 의한 열이다. 대기 중에 존재하는 산소는 분자 상태로 되어있다. 산소분자는 산소원자보다는 월등히 안전한 상태에 있다. 산소원자의 산화반응은 산소분자의 반응보다 엄청 빠르다. 만일, 공기중의 산소분자를 원자로 분리할 수만 있다면 물질을 산화시키는데 상당한 효과를 거둘 것으로 예상된다. 일반적으로 산소분자를 원자로 분리하는 것은 쉬운 일이 아니다. 그러나 플라즈마를 쓰면 쉽게 산소분자를 분리 할 수 있다.
최근 활발히 연구되고 있는 저온플라즈마 방식은 높은 고전압에 의해 전극 주위의 기체가 전기적으로 불안정하게 분해되어, 하전입자가 부분적으로 발생, 소멸을 반복하면서 방전 현상을 지속하는 상태이다. 일반적으로 전류의 발생은 작으며 가스 또는 이온의 온도는 낮지만 전자의 온도를 10,000∼100,000 K로 높게 유지된다. 이는 충돌을 통해 자유 라디칼을 생성시키는 전자들에 선택적으로 전기 에너지를 전달하여 원하는 화학 반응이 보다 잘 일어나도록 하기 위해서이다.
본 연구에서는 플라즈마 시스템에서 반응기 구조에 따른 코로나방전특성을 분석하기 위하여 음의 코로나(Cathode corona)와 양의 코로나(Anode corona)의 방전특성, 혼합형상의 방전특성, 전극간거리에 따른 방전특성 및 분위기가스에 따른 방전특성을 실험을 통하여 연구하였으며, 플라즈마에 의한 NO_(2) 생성을 최적화하기 위하여 전압변화, 전극변화 및 O_(2)의 농도에 따른 NO_(2)로의 전환특성을 알아보았다. 또한 NO_(2) 가스에 의해 Soot를 보다 효과적으로 저감시키고자 Soot의 산화 특성에 대하여 연구를 수행하였다. O_(2) 가스와 NO_(2) 가스에 의한 산화특성, 코로나 방전에 의해 생성된 NO_(2)에 의한 산화특성, 산소농도에 따른 산화특성, NO_(2), NO 및 O_(2) 가스에 의한 산화특성, 유량에 따른 산화특성 및 시간변화

#### 6-5. 수집한 정보를 바탕으로 DataFrame 만들기

In [36]:
pd.DataFrame(data = {'저자':data[0], '발행년도':data[1], '제목':data[2], '부가정보':data[3], 'Url':urls[0]}, index = [0])

,저자,발행년도,제목,부가정보,Url
0,박용성,2002,코로나방전을 이용한 NO_(2) 전환 및 디젤엔진 Soot 산화에 관한 기초 연구 ...,현재 인류의 주된 에너지원은 화석연료(C_(n)H_(m))를 대기중의 산소(O_(2...,http://www.riss.kr/search/detail/DetailView.do...


#### 7. 논문 정보 수집 과정 함수화 하기
앞선 예제를 기반으로 함수 속 ???와 공란을 채워주세요

In [38]:
def store(url):
    data=[]  #빈 리스트
    authur=""  #저자
    year=""  #연도
    title=""  #제목
    first_info=""  #부가정보
        
    #저자
    """ 저자 정보의 첫번째 항목을 xpath와 .text 메소드를 통해 구해주세요! """
    author=driver.find_element_by_xpath('//*[@id="thesisInfoDiv"]/div[2]/div[1]/ul/li[1]/div/p/a').text
        
    #연도
    """ 연도 정보의 첫번째 항목을 xpath와 .text 메소드를 통해 구해주세요! """
    year=driver.find_element_by_xpath('//*[@id="thesisInfoDiv"]/div[2]/div[1]/ul/li[4]/div/p').text
    
    #제목
    """ 제목 정보의 첫번째 항목을 xpath와 .text 메소드를 통해 구해주세요! """
    title=driver.find_element_by_xpath('//*[@id="thesisInfoDiv"]/div[1]/h3').text
    
    #부가정보    
    moreViews = driver.find_elements_by_class_name('moreView')

    for moreView in moreViews:
        moreView.send_keys(Keys.ENTER)
        sleep(0.1)

    try:
        """ 부가정보의 첫번째 항목을 xpath와 .text 메소드를 통해 구해주세요! """
        first_info=driver.find_element_by_xpath('//*[@id="additionalInfoDiv"]/div/div[1]/div[3]/p').text
        
    except:
        first_info = "부가정보가 없습니다."
        
    data.append(author)
    data.append(year)
    data.append(title)
    data.append(first_info)
    data.append(url)
    paper.append(data)

#### 8. 수집 및 저장
해당 코드는 채점을 위한 코드로 수정하시면 안됩니다!

In [39]:
paper=[]
for url in urls:
    driver.get(url)
    store(url)
    sleep(1)

driver.close()  # 드라이버 닫기

paper_info = pd.DataFrame(paper, columns=['저자', '발행년도', '제목', '부가정보', 'Url'])
paper_info.to_csv('./paper_info.csv', encoding='utf-8')
paper_info

,저자,발행년도,제목,부가정보,Url
0,박용성,2002,코로나방전을 이용한 NO_(2) 전환 및 디젤엔진 Soot 산화에 관한 기초 연구 ...,현재 인류의 주된 에너지원은 화석연료(C_(n)H_(m))를 대기중의 산소(O_(2...,http://www.riss.kr/search/detail/DetailView.do...
1,김성태,2021,코로나-19가 사회적 불안감에 미치는 영향 : - 게임 산업 종사자를 중심으로 - ...,2020년 2월 이후로 전 사람간의 접촉으로 전염되는 코로나19가 전 세계로 확산되...,http://www.riss.kr/search/detail/DetailView.do...
2,박석훈,2014,코로나방식 필터대전 조건에 따른 공기정화필터의 여과특성 = Filtration Ch...,"쾌적한 실내공간, 초정밀 제품 생산, 개인위생보호 등의 여러 목적으로 공기정화필터의...",http://www.riss.kr/search/detail/DetailView.do...
3,손주연,2021,코로나-19 발생 이후 가정간편식(HMR) 소비행태변화 및 만족도 조사,The purpose of this study is to provide basic ...,http://www.riss.kr/search/detail/DetailView.do...
4,정용철,2008,코로나 방전형 정전기 제거장치에서의 이온생성 최적화 조건에 관한 연구 = A Stu...,The purpose of this study is to analyze the op...,http://www.riss.kr/search/detail/DetailView.do...
5,이해인,2021,코로나19 백신 접종의도 설문조사 고찰 연구,2020년 1월 30일 코로나바이러스의 전세계적 확산으로 ‘국제적 공중보건 비상사태...,http://www.riss.kr/search/detail/DetailView.do...
6,정서혜,2021,코로나-19로 변화된 환경에 대한 미술치료사의 경험연구 : 포커스 그룹 인터뷰를 중...,본 연구는 포커스 그룹 인터뷰를 사용하여 코로나-19로 인하여 미술치료사가 현장에서...,http://www.riss.kr/search/detail/DetailView.do...
7,"ZENG, SHIYA",2021,코로나19 관련 가시성과 책임 보도가 감정과 낙인에 미치는 영향에 대한 연구,코로나19(COVID-19)는 제 6번째 ‘국제적 공중보건 비상사태’로써 세계적으로...,http://www.riss.kr/search/detail/DetailView.do...
8,경규희,2021,코로나19 상황에서 교사의 직무요구가 직무소진에 미치는 영향 : 코로나19에 대한 ...,본 연구의 목적은 코로나19 상황에서 국내 중등학교(중·고등학교 및 이에 상응하는 ...,http://www.riss.kr/search/detail/DetailView.do...
9,"Zhang, Yuqing",2021,코로나19 예방행위 지속의도에 영향을 미치는 요인 연구 : 건강신념모델(HBM)의 적용,사람들이 질병 혹은 위험의 특성을 지각하는 것에 따라 우리는 그들의 예방 행동을 예...,http://www.riss.kr/search/detail/DetailView.do...
